<a id="item31"></a>


In [1]:
#libraries import
import pandas as pd
import numpy as np

In [2]:
#load data
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
#number of data points
concrete_data.shape

(1030, 9)

In [4]:
#data basic stats
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
#check for nulls and if data is clean and ready to use
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### Split data into predictors and target


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
#data normalisation
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to _n_cols_ since we will need this number when building our network.


In [10]:
n_cols = predictors_norm.shape[1] # number of predictors
print(n_cols)

8


<a id="item1"></a>


<a id='item32'></a>


## Import Keras


#### Let's go ahead and import the Keras library


In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [14]:
from sklearn.model_selection import train_test_split

# Split the data
#x_train, x_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.33, shuffle= True)

mse_list = []
#run for 50 times
for i in range(50):
    # build the model
    model = regression_model()
    # fit the model
    # verbose: 0 for no logging to stdout, 1 for progress bar logging, 2 for one log line per epoch
    # use shuffle to true to enable splitting the data
    history = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=0, shuffle = True)
    mse_list.append(history.history['val_loss'][-1])

2021-10-08 15:35:53.693318: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-10-08 15:35:53.700523: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2095145000 Hz
2021-10-08 15:35:53.702074: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55fa1022db80 executing computations on platform Host. Devices:
2021-10-08 15:35:53.702124: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-10-08 15:35:53.869339: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not 

In [15]:
for idx, val in enumerate(mse_list):
    print(idx, val)

0 177.470202646595
1 138.95030800038558
2 112.13415870543051
3 146.98770492208044
4 105.76990876923101
5 110.67263569260878
6 117.68042925492074
7 136.4510620265331
8 181.72016106762933
9 124.31438362096891
10 147.14229650404846
11 153.67929269735095
12 139.64956146536522
13 144.2512707262749
14 147.93013677010643
15 137.1825741764797
16 137.82665808223985
17 100.33477274576823
18 137.7069346603838
19 154.84920765898374
20 122.07859380036882
21 169.6148472264361
22 127.64430436810244
23 147.1195788337189
24 139.44138395439074
25 124.48092478533007
26 142.92408885770632
27 126.98128383599439
28 162.34393243882263
29 120.88828704735222
30 146.98402503547544
31 168.8029636518855
32 145.41934875685806
33 123.64287260975267
34 150.2617171204206
35 111.46496288213143
36 132.55358165753313
37 157.6236493255714
38 157.44868967988344
39 120.61366662084092
40 160.0148128274961
41 153.43850787017723
42 190.76865221535888
43 105.34455241971803
44 116.90098571777344
45 126.94494357309681
46 134.506

In [16]:
print("mean mse_list=", np.mean(mse_list), "std mse_list", np.std(mse_list))

mean mse_list= 138.44832868903202 std mse_list 20.31941057247868


In [ ]:
# a question: mean mse_list= 428.2980590773401 std mse_list 501.07992890249454
# b question: mean mse_list= 382.0305239779277 std mse_list 105.47725809715753 
# c question: mean mse_list= 181.90792855667067 std mse_list 30.753559957711143
# d question: mean mse_list= 138.44832868903202 std mse_list 20.31941057247868

# in overall we notice an improvement in the loss function as we increase the number of layers and epochs in the ML model.
